# A classification of images of the game 'Rock, Paper, Scissors'

The goal of this notebook is to classify the images of the game 'Rock, Paper, Scissors' into three sets: Rock, Paper and Scissors. It will be done using a quite simple neural network algorithm (Feed Forward, one hidden layer), implemented with the Python package Torch.

## Etablishment of the system 

In [ ]:
import os, numpy, torch
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.utils.data.dataloader as dataloader
import torch.optim as optim
import matplotlib.pyplot as plt

from torch.utils.data import TensorDataset
from torch.autograd import Variable
from torchvision import transforms, datasets
from PIL import Image

SEED = 1

# Is CUDA available?
cuda = torch.cuda.is_available()

# For reproducibility
torch.manual_seed(SEED)

if cuda:
    torch.cuda.manual_seed(SEED)

## Data preparation

The aim of the next code is to fill the 'Test' folder, with 20% of the images of the 'Train' folder.

In [ ]:
%%bash
PW="./img"  # The current path

# If the Test folder is not empty then move all files in the Train folder
FE=$(find $PW/Test -name "*" -type f)
if [ -z "$FE" ]; then
    echo "Initial Test folder creation."
else
    echo "Test folder re-creation."
    T_IMG=$(find $PW/Test -name "*.jpg" -type f)  # A list of the tests images
    
    for f in $T_IMG; do
        mv $f $(echo $f | sed 's/Test/Train/')  # Move into Train directory
    done
fi

# Create the Test folder
IMG=$(find $PW/Train -name "*.jpg" -type f | shuf)  # A list of the images
LEN=$(echo -e $IMG | wc -w)  # The length of the IMG
T_RATE="0.2"  # The rate of test images

# A list of random FILES with theirs paths
FILES=$(echo -e $IMG | tr " " "\n" | tail -n $(echo $LEN*$T_RATE | bc | awk -F '.' '{ print $1 }'))

for f in $FILES; do
    mv $f $(echo $f | sed 's/Train/Test/')  # Move into Test directory
done


We can now import the data in the notebook and create the dataloader. However, for this first exemple, no data augmentation and complex preprocessing have been done. The main objective of this work is just to implement an operational algorithm.

In [ ]:
# Import the dataset

path = './img'
transform = transforms.Compose([
    transforms.Resize((32, 32)),
    transforms.Grayscale(),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(20, resample=Image.BILINEAR),
    transforms.ToTensor(), # ToTensor does min-max normalization.
])
train = datasets.ImageFolder(os.path.join(path,'Train'), transform=transform)
test = datasets.ImageFolder(os.path.join(path,'Test'), transform=transform)

# Create the dataloader
dataloader_args = dict(shuffle=True, batch_size=20, num_workers=4, pin_memory=True)
train_loader = dataloader.DataLoader(train, **dataloader_args)
dataloader_args = dict(shuffle=True, batch_size=len(test), num_workers=4, pin_memory=True)
test_loader = dataloader.DataLoader(test, **dataloader_args)

## The neural network algorithm 

In [ ]:
# Definition of the one-hidden-layer feed-forward network.
class Model(nn.Module):
    """
    Fully-connected model, with one hidden layer.
    
    Attributes
    ----------
    fc : torch.nn.modules.linear.Linear
        The first fully-connected layer.
    fc2 : torch.nn.modules.linear.Linear
        The second fully connected layer.
    
    Examples
    --------
    >>> model = Model()
    >>> if cuda:
    ...     model.cuda()
    >>>> optimizer = optim.Adam(model.parameters(), lr=1e-3)
    """
    
    def __init__(self):
        """
        Creation of the model.
        """
        super(Model, self).__init__()
        self.fc = nn.Linear(1024, 1000)  # The size of the images are 32*32 = 1024
        self.fc2 = nn.Linear(1000, 3)  # There are 3 differents classes

    def forward(self, x):
        """
        A forward pass in the model.
        
        Parameters
        ----------
        x : torch.nn_like
            The input of the model.
        
        Returns
        -------
        out : torch.nn_like
            The output of the model.
        
        Notes
        -----
        This function as not to be call directly.
        """
        x = x.view((-1, 1024))
        h = torch.relu(self.fc(x))
        h = self.fc2(h)
        return F.log_softmax(h, dim=0)    
    
    
model = Model()
if cuda:
    model.cuda()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

In [ ]:
EPOCHS = 15
losses = []

# Evaluation of the loader
evaluate = iter(test_loader)
evaluate_x, evaluate_y = evaluate.next()
if cuda:
    evaluate_x, evaluate_y = evaluate_x.cuda(), evaluate_y.cuda()

train_size = len(train_loader.dataset)
batch_size = 7  #(train_size / 256) if (cuda) else (train_size / 64)

model.train()
for epoch in range(EPOCHS):
    for batch_idx, (data, target) in enumerate(train_loader):
        # Get samples
        data, target = Variable(data), Variable(target)
        if cuda:
            data, target = data.cuda(), target.cuda()
        
        # Initialization
        optimizer.zero_grad()

        # Predict
        y_pred = model(data) 
        
        # Calculate loss
        loss = F.cross_entropy(y_pred, target)
        losses.append(loss.cpu().item())
        # Backpropagation
        loss.backward()
        optimizer.step()
        
        # Display
        if batch_idx % 100 == 1:
            print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch+1,
                EPOCHS,
                batch_idx * len(data), 
                train_size,
                100. * batch_idx / batch_size, 
                loss.cpu().item()), 
                end='')

    # Display final evaluation for this epoch
    model.eval()
    output = model(evaluate_x)
    pred = output.data.max(1)[1]
    d = pred.eq(evaluate_y.data).cpu()
    accuracy = d.sum().item()/d.size()[0]
    
    print('\r Train Epoch: {}/{} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\t Test Accuracy: {:.4f}%'.format(
        epoch+1,
        EPOCHS,
        train_size, 
        train_size,
        100. * batch_idx / batch_size, 
        loss.cpu().item(),
        accuracy*100,
        end=''))

The hyperparameters has been chosen almost randomly. For instance, 15 epochs are enough to test the validity of our network : There are 3 classes and the test accuracy is greater than 33%.

In [ ]:
plt.plot(losses)